In [1]:
import json
import os
import random

os.chdir("/mnt/bn/nlhei-nas/liubangya/proj/vlm/QA")
print(f"current working dir", os.getcwd())

# function pool here
from templates_lib.filter import *
from templates_lib.func import *
from templates_lib.QA import QADataset

current working dir /mnt/bn/nlhei-nas/liubangya/proj/vlm/QA


### Static::Measurement::object_distance

In [2]:
DS_ROOT = "./structured-data"
TEST_SPLIT = 0.2
CAP_FILE = "captions_all.yaml"

random.seed(0)
myCap = Captioner(CAP_FILE)
ds = QADataset(DS_ROOT, myCap)


  0%|          | 0/850 [00:00<?, ?it/s]

In [6]:
from templates_lib.task import MultiTaskSet
from templates_lib.task.distance_tasks import DistTasks
from templates_lib.task.movement_tasks import MovementTasks
from templates_lib.task.grid_tasks import GridIndexTasks
from templates_lib.task.tracking_tasks import TrackingTasks




tasks_cfg = {
    "total_QAs": 6000,
    "roi_frame_only": True,
    "H": 900, "W": 1600, "patchsize_H": 100, "patchsize_W": 100,
    "motion_thres": 4, "num_frame": 4
}


total_qas = tasks_cfg["total_QAs"]
H_grids = tasks_cfg["H"] // tasks_cfg["patchsize_H"]
W_grids = tasks_cfg["W"] // tasks_cfg["patchsize_W"]
# task_dir = f"/mnt/bn/nlhei-nas/liubangya/proj/vlm-found3d/tasks/grid_idx_{H_grids}x{W_grids}"
task_dir = f"/mnt/bn/nlhei-nas/liubangya/proj/vlm-found3d/tasks/image_nohint_{H_grids}x{W_grids}_{tasks_cfg['num_frame']}"
OUTPUT_DIR = os.path.join(task_dir, "pairs")
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_QWEN = f"{OUTPUT_DIR}/QA_pairs_qwen.json"
OUTPUT_JSON = f"{OUTPUT_DIR}/QA_pairs.json"

task_obj = {
    os.path.basename(task_dir): {
        "train_qa": OUTPUT_QWEN.replace(".json", ".train.json"),
        "test_qa": OUTPUT_QWEN.replace(".json", ".test.json"),
        "train_qa_meta": OUTPUT_JSON.replace(".json", ".train.json"),
        "test_qa_meta": OUTPUT_JSON.replace(".json", ".test.json"),
    }
}
print(json.dumps(task_obj, indent=4))


{
    "image_nohint_9x16_4": {
        "train_qa": "/mnt/bn/nlhei-nas/liubangya/proj/vlm-found3d/tasks/image_nohint_9x16_4/pairs/QA_pairs_qwen.train.json",
        "test_qa": "/mnt/bn/nlhei-nas/liubangya/proj/vlm-found3d/tasks/image_nohint_9x16_4/pairs/QA_pairs_qwen.test.json",
        "train_qa_meta": "/mnt/bn/nlhei-nas/liubangya/proj/vlm-found3d/tasks/image_nohint_9x16_4/pairs/QA_pairs.train.json",
        "test_qa_meta": "/mnt/bn/nlhei-nas/liubangya/proj/vlm-found3d/tasks/image_nohint_9x16_4/pairs/QA_pairs.test.json"
    }
}


In [7]:

myfilter = filter_all(
    filter_visiblity,
    filter_area_fn(1e4, 4e5),
    black_list_fn([
            "movable_object.trafficcone",
            "movable_object.barrier",
        ])
    )

# taskset = MultiTaskSet(
#     subsets=[DistTasks, MovementTasks],
#     captioner=myCap,
#     basefilter=myfilter,
#     cfg=tasks_cfg)

# taskset = GridIndexTasks(
#     H = 900, W = 1600, patchsize_H = 100, patchsize_W = 100,
#     captioner=myCap,
#     basefilter=myfilter,
#     cfg=tasks_cfg
# )

taskset = TrackingTasks(
    captioner=myCap,
    basefilter=myfilter,
    cfg=tasks_cfg
)


qas, stats = taskset.produce(
    dataset=ds,
    num_qas=total_qas,
    # verbose=True
)

print(f"total {len(qas)} qas")
print(f"stats: {json.dumps(stats, indent=2)}")

all_dumps = [qa.dump() for qa in qas]
content_stats = {
    "objs": set(),
    "scenes": set(),
}
for qa in all_dumps:
    content_stats["objs"].update(qa["objs"])
    content_stats["scenes"].update([qa["scene"]])
print(f"total objects: {len(content_stats['objs'])}")
print(f"total scenes: {len(content_stats['scenes'])}")



Generated 500 QAs, stats: {'obj_cross_frame_tracking': 500}
Generated 1000 QAs, stats: {'obj_cross_frame_tracking': 1000}
Generated 1500 QAs, stats: {'obj_cross_frame_tracking': 1500}
Generated 2000 QAs, stats: {'obj_cross_frame_tracking': 2000}
Generated 2500 QAs, stats: {'obj_cross_frame_tracking': 2500}
Generated 3000 QAs, stats: {'obj_cross_frame_tracking': 3000}
Generated 3500 QAs, stats: {'obj_cross_frame_tracking': 3500}
Generated 4000 QAs, stats: {'obj_cross_frame_tracking': 4000}
Generated 4500 QAs, stats: {'obj_cross_frame_tracking': 4500}
Generated 5000 QAs, stats: {'obj_cross_frame_tracking': 5000}
Generated 5500 QAs, stats: {'obj_cross_frame_tracking': 5500}
Generated 6000 QAs, stats: {'obj_cross_frame_tracking': 6000}
total 6000 qas
stats: {
  "obj_cross_frame_tracking": 6000
}
total objects: 2707
total scenes: 759


In [8]:

num_test = int(len(qas) * TEST_SPLIT)
qas_train = qas[:-num_test]
qas_test = qas[-num_test:]

all_frames = not taskset.cfg["roi_frame_only"]

with open(OUTPUT_QWEN.replace(".", ".test."), "w") as f:
    qas_dumps = [qa.qwen_format(all_frames=all_frames) for qa in qas_test]
    for i, qa in enumerate(qas_dumps):
        qa["id"] = i
    json.dump(
        qas_dumps, f, indent=2
    )
with open(OUTPUT_QWEN.replace(".", ".train."), "w") as f:
    qas_dumps = [qa.qwen_format(all_frames=all_frames) for qa in qas_train]
    for i, qa in enumerate(qas_dumps):
        qa["id"] = i
    json.dump(
        qas_dumps, f, indent=2
    )
with open(OUTPUT_JSON.replace(".", ".test."), "w") as f:
    qas_dumps = [qa.dump() for qa in qas_test]
    for i, qa in enumerate(qas_dumps):
        qa["id"] = i
    json.dump(qas_dumps, f, indent=2)
with open(OUTPUT_JSON.replace(".", ".train."), "w") as f:
    qas_dumps = [qa.dump() for qa in qas_train]
    for i, qa in enumerate(qas_dumps):
        qa["id"] = i
    json.dump(qas_dumps, f, indent=2)
